In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
model_path = "./FlagAlpha--Llama2-Chinese-7b-Chat/"

tokenizer = AutoTokenizer.from_pretrained(model_path,use_fast=False)
# tokenizer.pad_token = tokenizer.eos_token

In [4]:
tokenizer.bos_token_id

1

In [6]:
tokenizer.padding_side

'right'

In [10]:
tokenizer("<s>hello", add_special_tokens=False)

{'input_ids': [1, 12199], 'attention_mask': [1, 1]}

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16)
model = model.eval();
model.to("cuda:0");

[2023-10-20 16:44:48,646] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/wyd/miniconda3/envs/flexgen/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/wyd/miniconda3/envs/flexgen/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [4]:
for key,val in model.named_parameters():
    print(key,val.shape,val.dtype)

model.embed_tokens.weight torch.Size([32000, 4096]) torch.float16
model.layers.0.self_attn.q_proj.weight torch.Size([4096, 4096]) torch.float16
model.layers.0.self_attn.k_proj.weight torch.Size([4096, 4096]) torch.float16
model.layers.0.self_attn.v_proj.weight torch.Size([4096, 4096]) torch.float16
model.layers.0.self_attn.o_proj.weight torch.Size([4096, 4096]) torch.float16
model.layers.0.mlp.gate_proj.weight torch.Size([11008, 4096]) torch.float16
model.layers.0.mlp.up_proj.weight torch.Size([11008, 4096]) torch.float16
model.layers.0.mlp.down_proj.weight torch.Size([4096, 11008]) torch.float16
model.layers.0.input_layernorm.weight torch.Size([4096]) torch.float16
model.layers.0.post_attention_layernorm.weight torch.Size([4096]) torch.float16
model.layers.1.self_attn.q_proj.weight torch.Size([4096, 4096]) torch.float16
model.layers.1.self_attn.k_proj.weight torch.Size([4096, 4096]) torch.float16
model.layers.1.self_attn.v_proj.weight torch.Size([4096, 4096]) torch.float16
model.layer

In [20]:
4096*4096*3 /(512*(4096*3)+(4096*3))

7.984405458089668

In [6]:
input_ids = tokenizer(['<s>Human: 介绍一下中国\n</s><s>Assistant: '], return_tensors="pt",add_special_tokens=False).input_ids.to('cuda')        
generate_input = {
    "input_ids":input_ids,
    "max_new_tokens":512,
    "do_sample":True,
    "top_k":50,
    "top_p":0.95,
    "temperature":0.3,
    "repetition_penalty":1.3,
    "eos_token_id":tokenizer.eos_token_id,
    "bos_token_id":tokenizer.bos_token_id,
    "pad_token_id":tokenizer.pad_token_id
}

In [7]:
generate_ids  = model.generate(**generate_input)
text = tokenizer.decode(generate_ids[0])
print(text)

/home/wyd/miniconda3/envs/flexgen/lib/python3.8/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<s>Human: 介绍一下中国
</s><s>Assistant: 作为世界上最大的人口和面积，历史悠久、文化丰富的中华民族在数千年来不断发展成长。现代社会中，我们正处于全球经济趋同工程时期，需要更加注重科技创新与高效管理等方面的实际运用能力。建设可持续性是当前所有行业都必须关心的问题之一，而对这个话题也提出了相应的解决思路。首先，通过改进生态保护政策并加速节能转变以及自然资源利用的深度开放；其次，加强基本教育水平和就业机会的广泛分配；再者，引导消费主体向健康、安全、环保三位一体的品牌选择，从而形成市场支持良好的商家群体。此外还包括多种合法手段如公共服务活动、志愿者组织参与、网络传播等，希望每一个人都能认真反省自身，到达“小事”的意义，给城乡居民增添美好的日常生活乐趣！
</s>


In [10]:
# ./main -m ggml_output/ggml-model-q4_0.gguf -p "<s>Human: 介绍一下中国\n</s><s>Assistant: "

In [1]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model = AutoGPTQForCausalLM.from_quantized(
    "./gptq_output/",
    device="cuda:1", 
)

[2023-10-27 15:36:45,535] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.


In [8]:
model.model.model.layers[0].self_attn.qkv_proj.weight

Parameter containing:
tensor([[-2019129257, -2004318071, -2004318072,  ...,  1756867686,
           678656169,  1537702536],
        [-2006349400, -1986492536, -2022143864,  ..., -1759015642,
         -1235646888,  2008529074],
        [ 2054715255, -1853319016, -1886877576,  ..., -1734820422,
          1478080180,  1486316939],
        ...,
        [-1847615302, -1478832996,     5714507,  ...,  2009618828,
            -9982040,  -929388452],
        [  469758001, -1510361463,  -977887862,  ..., -1392946315,
          1829550008,  1035705504],
        [ -258158699,  -146814948,  -930760287,  ...,  -645957253,
           259539631, -1783313018]], device='cuda:1', dtype=torch.int32)

In [3]:
model

LlamaGPTQForCausalLM(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096, padding_idx=0)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): FusedLlamaAttentionForQuantizedModel(
            (qkv_proj): GeneralQuantLinear(in_features=4096, out_features=12288, bias=True)
            (o_proj): GeneralQuantLinear(in_features=4096, out_features=4096, bias=True)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (act_fn): SiLUActivation()
            (down_proj): GeneralQuantLinear(in_features=11008, out_features=4096, bias=True)
            (gate_proj): GeneralQuantLinear(in_features=4096, out_features=11008, bias=True)
            (up_proj): GeneralQuantLinear(in_features=4096, out_features=11008, bias=True)
          )
          (input_layernorm): LlamaRMSNorm()
          (post_attention_layernorm): LlamaRMSNorm()
        )
      )
      (norm): LlamaRMSN

In [2]:
for key,val in model.named_parameters():
    print(key,val.shape,val.dtype)

model.model.embed_tokens.weight torch.Size([32000, 4096]) torch.float16
model.model.layers.0.self_attn.qkv_proj.weight torch.Size([512, 12288]) torch.int32
model.model.layers.0.self_attn.qkv_proj.bias torch.Size([12288]) torch.float16
model.model.layers.0.self_attn.o_proj.weight torch.Size([512, 4096]) torch.int32
model.model.layers.0.self_attn.o_proj.bias torch.Size([4096]) torch.float16
model.model.layers.0.mlp.down_proj.weight torch.Size([1376, 4096]) torch.int32
model.model.layers.0.mlp.down_proj.bias torch.Size([4096]) torch.float16
model.model.layers.0.mlp.gate_proj.weight torch.Size([512, 11008]) torch.int32
model.model.layers.0.mlp.gate_proj.bias torch.Size([11008]) torch.float16
model.model.layers.0.mlp.up_proj.weight torch.Size([512, 11008]) torch.int32
model.model.layers.0.mlp.up_proj.bias torch.Size([11008]) torch.float16
model.model.layers.0.input_layernorm.weight torch.Size([4096]) torch.float16
model.model.layers.0.post_attention_layernorm.weight torch.Size([4096]) torch

In [ ]:
print(tokenizer.decode(model.generate(
    **tokenizer("<s>Human: 介绍一下中国\n</s><s>Assistant: ", return_tensors="pt").to(model.device))[0]))